In [1]:
import sys
sys.path.append('..')
import utils
batch_size = 256
train_data, test_data = utils.load_data_fashion_mnist(batch_size)

/home/dyjng/anaconda3/lib/python3.6/site-packages/urllib3/contrib/pyopenssl.py:46: DeprecationWarning: OpenSSL.rand is deprecated - you should use os.urandom instead
  import OpenSSL.SSL


In [2]:
from mxnet import ndarray as nd

num_inputs = 28 * 28
num_outputs = 10

num_hidden1 = 256
num_hidden2 = 64
weight_scale = .01

w1 = nd.random_normal(shape = (num_inputs, num_hidden1), scale = weight_scale)
b1 = nd.zeros(num_hidden1)

w2 = nd.random_normal(shape = (num_hidden1, num_hidden2), scale = weight_scale)
b2 = nd.zeros(num_hidden2)

w3 = nd.random_normal(shape = (num_hidden2, num_outputs), scale = weight_scale)
b3 = nd.zeros(num_outputs)

params = [w1, b1, w2, b2, w3, b3]

for param in params:
    param.attach_grad()

In [3]:
def relu(X):
    return nd.maximum(X, 0)

In [4]:
def net(X):
    X = X.reshape((-1, num_inputs))
    h1 = relu(nd.dot(X, w1) + b1)
    h2 = relu(nd.dot(h1, w2) + b2)
    output = nd.dot(h2, w3) + b3
    return output

In [5]:
from mxnet import gluon
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()

In [6]:
from mxnet import autograd as ag

learning_rate = .5

for epoch in range(5):
    train_loss = 0
    train_acc = 0
    for data, label in train_data:
        with ag.record():
            output = net(data)
            loss = softmax_cross_entropy(output, label)
        loss.backward()
        utils.SGD(params, learning_rate / batch_size)
        
        train_loss += nd.mean(loss).asscalar()
        train_acc += utils.accuracy(output, label)
    test_acc = utils.evaluate_accuracy(test_data, net)
    print('Epoch %d, Loss: %f, Train acc %f, Test acc %f' % (
        epoch, train_loss / len(train_data), train_acc / len(train_data), test_acc))

Epoch 0, Loss: 1.156641, Train acc 0.537861, Test acc 0.737780
Epoch 1, Loss: 0.564100, Train acc 0.787760, Test acc 0.814002
Epoch 2, Loss: 0.467904, Train acc 0.825938, Test acc 0.834535
Epoch 3, Loss: 0.419070, Train acc 0.844050, Test acc 0.833834
Epoch 4, Loss: 0.395328, Train acc 0.853849, Test acc 0.865184


In [7]:
print(params)

[
[[ 0.00299947  0.00234668  0.01049332 ..., -0.01590716 -0.01479646
  -0.01379859]
 [ 0.00910866 -0.01326521 -0.01155486 ...,  0.00138668 -0.01497074
   0.01851147]
 [-0.00643633  0.01019931  0.00363147 ...,  0.00773294 -0.01196061
   0.00242589]
 ..., 
 [-0.00924341 -0.01247925 -0.00232519 ..., -0.01805013  0.01537773
   0.00267385]
 [-0.00298401 -0.00762905 -0.01573109 ..., -0.00851855  0.00155496
  -0.00235174]
 [ 0.00789364 -0.00335047  0.01933927 ...,  0.00784953 -0.00233528
   0.00188258]]
<NDArray 784x256 @cpu(0)>, 
[  8.70409384e-02   7.47765526e-02  -1.13227144e-01   1.24931157e-01
   5.57297617e-02   1.11301064e-01   1.71914920e-02   3.62242758e-02
   2.31300686e-02   1.06338628e-01  -4.55473736e-03   2.27707818e-01
   1.34926224e-02   8.40333849e-03   2.35370342e-02   1.47900730e-02
   1.22069577e-02   4.98593003e-02   1.53220231e-02   8.06900039e-02
  -2.43924353e-02   1.05507657e-01  -7.11901113e-02   7.60322586e-02
   7.05406591e-02   2.01334935e-02  -3.63398865e-02  -5.